<a href="https://colab.research.google.com/github/cesarvian/Wolbachia/blob/main/Optimizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 0. Instalar paquetes ----

# Cargar las librerías necesarias
if (!require("pacman")) {
  install.packages("pacman", dependencies = TRUE)
  library("pacman")
}

pacman::p_load(mco, # Algoritmo genetico GA
               rio,
               deSolve,  # Simulación numérica
               ecr, # Auxiliar para GA
               # Algoritmo genetico GA
               tidyverse, # gramatica de manipulación de dataframe
               ggplot2,# Gramatica para objetos gg de visualización
               parallel,
               doParallel,
               devtools,
               reshape2,
               gganimate,
               cdata)

# 1. Definir modelo ----

# Definir función de modelo x'=f(x,u)

TSW_Model = function(times, x, parms, input)  {
  with(as.list(c(parms, x)), {
    ## Auxiliar
    # Vector and Matrix
    x_M = c(x_UM, x_WM)
    x_F = c(x_UF, x_WF)
    x   = c(x_UM, x_UF, x_WM, x_WF)
    e_4_1 = c(1, 0, 0, 0)
    e_4_2 = c(0, 1, 0, 0)
    e_4_3 = c(0, 0, 1, 0)
    e_4_4 = c(0, 0, 0, 1)
    e_2_1 = c(1, 0)
    e_2_2 = c(0, 1)
    o_4 = e_4_1 + e_4_2 + e_4_3 + e_4_4
    # o_4a = e_4_1*a_U + e_4_2 + e_4_3*a_U + e_4_4
    o_2 = e_2_1 + e_2_2
    G_U = matrix(c(1,  0,
                   1 - sigma, 0), nrow = 2, byrow = TRUE)
    G_W = matrix(c(0,  1,
                   0, 1), nrow = 2, byrow = TRUE)
    # Función bilinear de nacimientos
    b_U = (t(x_M) %*% G_U %*% x_F) / t(o_4) %*% x
    b_W = (t(x_M) %*% G_W %*% x_F) / t(o_4) %*% x
    # Operador de nacimientos
    Bx = c(b_U, b_U, b_W, b_W)
    # Seasonality of average temperature or constant
    s = s_seas * (A + B * cos(2 * pi * times / 365) + C * sin(2 * pi * times /
                                                                365)) + s_const * rep(25, length(times))
    #  the time-dependent oviposition rate mUM0_t, mUF0_t, mWM0_t, mWF0_t
    mUM0_t = mUM0_c * s * (s - mUM0_T0) * (mUM0_Tm - s) ^ (1 / 2)
    mUF0_t = mUF0_c * s * (s - mUF0_T0) * (mUF0_Tm - s) ^ (1 / 2)
    mWM0_t = (1 - m_w) * mWM0_c * s * (s - mWM0_T0) * (mWM0_Tm - s) ^ (1 /
                                                                         2)
    mWF0_t = (1 - m_w) * mWF0_c * s * (s - mWF0_T0) * (mWF0_Tm - s) ^ (1 /
                                                                         2)
    #
    mUM_t =  mUM0_t - mUM * t(o_4) %*% x
    mUF_t =  mUF0_t  - mUF * t(o_4) %*% x
    mWM_t =  mWM0_t - mWM * t(o_4) %*% x
    mWF_t =  mWF0_t - mWF * t(o_4) %*% x
    # Matriz diagonal reclutamiento
    m = matrix(
      c(mUM_t, 0, 0, 0,
        0, mUF_t, 0, 0,
        0, 0, mWM_t, 0,
        0, 0, 0, mWF_t),
      nrow = 4,
      byrow = TRUE
    )
    # time-dependent death rates of adult mosquitoes muUM0_t, muUF0_t, muWM0_t, muWF0_t
    muUM0_t = (muUM0_c * (s - muUM0_T0) * (muUM0_Tm - s)) ^ (-1)
    muUF0_t = (muUF0_c * (s - muUF0_T0) * (muUF0_Tm - s)) ^ (-1)
    muWM0_t = (1 + mu_w) * (muWM0_c * (s - muWM0_T0) * (muWM0_Tm - s)) ^
      (-1)
    muWF0_t = (1 + mu_w) * (muWF0_c * (s - muWF0_T0) * (muWF0_Tm - s)) ^
      (-1)
    muUM_t = muUM0_t + muUM * t(o_4) %*% x
    muUF_t = muUF0_t + muUM * t(o_4) %*% x
    muWM_t = muWM0_t + muUM * t(o_4) %*% x
    muWF_t = muWF0_t + muUM * t(o_4) %*% x
    # Matriz diagonal mortalidad
    mu = matrix(
      c(muUM_t, 0, 0, 0,
        0, muUF_t, 0, 0,
        0, 0, muWM_t, 0,
        0, 0, 0, muWF_t),
      nrow = 4,
      byrow = TRUE
    )
    # Import impulsive or const interval
    WM_t = WM * (impul * ifelse(-1 + 1 * cos(2 * pi * times / dd) < 0, 0, 1) + cont *
                   (times >= lowerM & times <= upperM))  # Machos infectados
    WF_t = WF * (impul * ifelse(-1 + 1 * cos(2 * pi * times / dd) < 0, 0, 1) + cont *
                   (times >= lowerF & times <= upperF)) # Hembras infectadass
    W = c(0, 0, WM_t, WF_t)
    # ODE System
    dx = m %*% Bx - mu %*% x + W
    res <- dx
    return(
      list(
        c(dx),
        s = s,
        WM_t = WM_t,
        WF_t = WF_t,
        mUM0_t = mUM0_t,
        mUF0_t = mUF0_t,
        mWM0_t = mWM0_t,
        mWF0_t = mWF0_t,
        muUM0_t = muUM0_t,
        muUF0_t = muUF0_t,
        muWM0_t = muWM0_t,
        muWF0_t = muWF0_t
      )
    )
  })
}

# 2. Definir función de fitness ----

# Definir la función de ajuste con los nuevos parámetros de liberación
fitness_function <- function(X, ...) {
  # Calcular los parámetros que se están optimizando
  parms_temp <- c(
    sigma = 0.95,
    # coeficiente de incompatibilidad citoplasmatica
    #recruitment reduction ratio due to infection
    m_w = 0.350,
    # proportion of increased mortality due to infection
    mu_w = 0.350,
    #  the time-dependent oviposition rate parameters
    mUM0_c = 0.0058,
    mUF0_c = 0.0058,
    mWM0_c = 0.0058,
    mWF0_c = 0.0058,
    mUM0_T0 = 14.0343,
    mUF0_T0 = 14.0343,
    mWM0_T0 = 14.0343,
    mWF0_T0 = 14.0343,
    mUM0_Tm = 39.0899,
    mUF0_Tm = 39.0899,
    mWM0_Tm = 39.0899,
    mWF0_Tm = 39.0899,
    mUM = 0.0001,
    mUF = 0.0001,
    mWM = 0.0001,
    mWF = 0.0001,
    # time-dependent death rates of adult mosquitoes parameters
    muUM0_c = 0.1037,
    muUF0_c = 0.1037,
    muWM0_c = 0.1037,
    muWF0_c = 0.1037,
    muUM0_T0 = 6.4429,
    muUF0_T0 = 6.4429,
    muWM0_T0 = 6.4429,
    muWF0_T0 = 6.4429,
    muUM0_Tm = 41.4382,
    muUF0_Tm = 41.4382,
    muWM0_Tm = 41.4382,
    muWF0_Tm = 41.4382,
    muUM = 0.0002,
    muUF = 0.0002,
    muWM = 0.0002,
    muWF = 0.0002,
    WM = 4000 * X[1],
    #  WM = 4000*X[1] # Cantidad de machos a introducir en el tiempo ¡Parametro a sintonizar!
    WF = 4000 * X[2],
    #   WF = 4000*X[2] # Parametro a sintonizar, cantidad de hembras a introducir en el tiempo ¡Parametro a sintonizar!
    dd = 7,
    # intervalos en días para liberación por impulso
    lowerM = X[3],
    # lowerM = 510, #  lowerM = X[3] # tiempo inicial de liberación de M ¡Parametro a sintonizar!
    upperM = X[3] + X[4],
    # upperM = 510+14, # upperM = X[3]+X[4] # tiempo final de liberación de M ¡Parametro a sintonizar!
    lowerF = X[3]+X[5],
    # lowerF = 520, #lowerF = X[5]  # tiempo inicial de liberación de F ¡Parametro a sintonizar!
    upperF = X[3]+X[5] + X[6],
    # upperF = 520+30, # upperF = X[5] + X[6]  # tiempo final de liberación de F ¡Parametro a sintonizar!
    A = 22.6071877371859,
    B = 4.67698314471263,
    C = 1.08897678248751,
    s_const = 0,
    s_seas = 1,
    impul = 0,
    cont = 1
  )

  ## vector of timesteps
  times = seq(0, 365 * 3)

  ## external signal with rectangle impulse
  signal = data.frame(times = times,
                      s = rep(NA, length(times)))


  ## Start values for steady state
  xstart <- c(
    x_UM = 4000,
    x_UF = 4000,
    x_WM = 0,
    x_WF = 0
  )

  ## Solve model
  out <- ode(
    y = xstart,
    times = times,
    func = TSW_Model,
    parms = parms_temp
  )

  out = (abs(out) + out) / 2

  ## Default plot method
  #plot(out)

  max_t = nrow(out)

  #obj_val1 = ((round(out[max_t, 2], 3) + 1) / (round(out[max_t, 4], 3) + 1)) #Calidad
  #obj_val2 = (parms_temp[4] + 1) * (4000 * X[1]) + (parms_temp[6] + 1) * (4000 * X[2]) #Costo
  #obj_val3 = ((out[max_t,2]+1)/(out[max_t,4]+1))*((parms_temp[4]+1)*(4000*X[1]) + (parms_temp[6]+1)*(4000*X[2]))
  poblacion_silvestre <- out[max_t, 3]  # x_UF
  poblacion_wolbachia <- out[max_t, 5]  # x_WF

  #costo <- (X[4]*(4000 * X[1])) + (X[6]*(4000 * X[2]))

  obj_val1 <- ((poblacion_silvestre + 1) / (poblacion_wolbachia + 1)) + (X[4] * (4000 * X[1]))

  obj_val2 <- ((poblacion_silvestre + 1) / (poblacion_wolbachia + 1)) + (X[6] * (4000 * X[2]))



  # Maximiza cambiando el simbolo al primero objetivo
  return(c(obj_val1, obj_val2))  # devuelve un vector de valores de ajuste
}
# Función auxiliar corregida (Faltaban parámetros muUM, muUF...)
calc_wolbachia_final <- function(X) {
  parms_temp <- c(
    sigma = 0.95, m_w = 0.350, mu_w = 0.350,
    mUM0_c = 0.0058, mUF0_c = 0.0058, mWM0_c = 0.0058, mWF0_c = 0.0058,
    mUM0_T0 = 14.0343, mUF0_T0 = 14.0343, mWM0_T0 = 14.0343, mWF0_T0 = 14.0343,
    mUM0_Tm = 39.0899, mUF0_Tm = 39.0899, mWM0_Tm = 39.0899, mWF0_Tm = 39.0899,
    mUM = 0.0001, mUF = 0.0001, mWM = 0.0001, mWF = 0.0001,
    muUM0_c = 0.1037, muUF0_c = 0.1037, muWM0_c = 0.1037, muWF0_c = 0.1037,
    muUM0_T0 = 6.4429, muUF0_T0 = 6.4429, muWM0_T0 = 6.4429, muWF0_T0 = 6.4429,
    muUM0_Tm = 41.4382, muUF0_Tm = 41.4382, muWM0_Tm = 41.4382, muWF0_Tm = 41.4382,

    # --- ESTAS 4 LINEAS FALTABAN ---
    muUM = 0.0002,
    muUF = 0.0002,
    muWM = 0.0002,
    muWF = 0.0002,
    # -------------------------------

    WM = 4000 * X[1],
    WF = 4000 * X[2],
    dd = 7,
    lowerM = X[3],
    upperM = X[3] + X[4],
    lowerF = X[3] + X[5],
    upperF = X[3] + X[5] + X[6],
    A = 22.6071877371859, B = 4.67698314471263, C = 1.08897678248751,
    s_const = 0, s_seas = 1, impul = 0, cont = 1
  )

  times = seq(0, 365 * 3)
  xstart <- c(x_UM = 4000, x_UF = 4000, x_WM = 0, x_WF = 0)
  out <- ode(y = xstart, times = times, func = TSW_Model, parms = parms_temp)
  out = (abs(out) + out) / 2
  max_t = nrow(out)

  return(c(WF = out[max_t, 5], UF = out[max_t, 3]))
}

# ============================================================================
# SISTEMA DE OPTIMIZACIÓN INCREMENTAL CON GUARDADO Y GRÁFICOS
# ============================================================================



# Configuración inicial
popsize <- 100
total_generations <- 100
increment <- 10
results_history <- list()
time_history <- list()
convergence_data <- data.frame()

# Crear carpeta para resultados si no existe
if (!dir.exists("resultados_optimizacion")) {
  dir.create("resultados_optimizacion")
}

save_cycle_results <- function(result, cycle, current_generations) {

  # 1. Guardar convergencia (Sin cambios)
  cycle_data <- data.frame(
    ciclo = cycle,
    generacion_inicio = current_generations - increment,
    generacion_fin = current_generations,
    tiempo_ejecucion = as.numeric(Sys.time() - start_time),
    mejores_obj1 = min(result@fitness[,1]),
    mejores_obj2 = min(result@fitness[,2]),
    num_soluciones = nrow(result@fitness),
    fecha = Sys.time()
  )

  write.table(cycle_data,
              "resultados_optimizacion/convergence_history.csv",
              append = file.exists("resultados_optimizacion/convergence_history.csv"),
              sep = ",", col.names = !file.exists("resultados_optimizacion/convergence_history.csv"),
              row.names = FALSE)

  # 2. Calcular WF y UF
  # apply devuelve una matriz donde fila 1 es WF y fila 2 es UF. Usamos t() para transponer.
  val_pob <- t(apply(result@population, 1, calc_wolbachia_final))

  # 3. Crear Dataframe de Soluciones
  solutions_data <- data.frame(
    ciclo = cycle,
    generacion = current_generations,
    result@population,
    obj1 = result@fitness[,1],
    obj2 = result@fitness[,2],
    poblacion_WF = val_pob[,1],                        # Columna nueva 1: Wolbachia
    poblacion_UF = val_pob[,2],                        # Columna nueva 2: Silvestres (UF)
    estado = ifelse(val_pob[,1] > 4591, "Exito", "Fracaso") # Umbral ajustado a 400
  )

  # Renombrar columnas de parámetros
  colnames(solutions_data)[3:8] <- c("WM_factor", "WF_factor", "tiempo_inicio",
                                     "duracion_M", "delay_F", "duracion_F")

  write.table(solutions_data,
              "resultados_optimizacion/solutions_history.csv",
              append = file.exists("resultados_optimizacion/solutions_history.csv"),
              sep = ",", col.names = !file.exists("resultados_optimizacion/solutions_history.csv"),
              row.names = FALSE)

  return(cycle_data)
}
# Función para graficar evolución de la convergencia
plot_convergence <- function() {
  if (file.exists("resultados_optimizacion/convergence_history.csv")) {
    conv_data <- read.csv("resultados_optimizacion/convergence_history.csv")

    p1 <- ggplot(conv_data, aes(x = generacion_fin)) +
      geom_line(aes(y = mejores_obj1, color = "Objetivo 1"), size = 1) +
      geom_line(aes(y = mejores_obj2, color = "Objetivo 2"), size = 1) +
      geom_point(aes(y = mejores_obj1, color = "Objetivo 1"), size = 2) +
      geom_point(aes(y = mejores_obj2, color = "Objetivo 2"), size = 2) +
      labs(title = "Evolución de los Mejores Valores por Ciclo",
           x = "Generación", y = "Valor de Fitness",
           color = "Función Objetivo") +
      theme_minimal() +
      scale_color_manual(values = c("Objetivo 1" = "blue", "Objetivo 2" = "red"))

    p2 <- ggplot(conv_data, aes(x = generacion_fin, y = num_soluciones)) +
      geom_line(color = "darkgreen", size = 1) +
      geom_point(color = "darkgreen", size = 2) +
      labs(title = "Número de Soluciones No Dominadas por Ciclo",
           x = "Generación", y = "Número de Soluciones") +
      theme_minimal()

    # Guardar gráficos
    ggsave("resultados_optimizacion/evolucion_fitness.png", p1, width = 10, height = 6)
    ggsave("resultados_optimizacion/evolucion_soluciones.png", p2, width = 10, height = 6)

    # Mostrar en R
    print(p1)
    print(p2)
  }
}

# Función para graficar frentes de Pareto por ciclo
plot_pareto_fronts <- function() {
  if (file.exists("resultados_optimizacion/solutions_history.csv")) {
    sol_data <- read.csv("resultados_optimizacion/solutions_history.csv")

    p <- ggplot(sol_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
      geom_point(alpha = 0.7, size = 2) +
      labs(title = "Evolución del Frente de Pareto por Ciclo",
           x = "Función Objetivo 1",
           y = "Función Objetivo 2",
           color = "Ciclo") +
      theme_minimal() +
      scale_color_viridis_d()

    ggsave("resultados_optimizacion/frentes_pareto_evolucion.png", p, width = 10, height = 6)
    print(p)
  }
}

# Función para graficar evolución de parámetros
plot_parameters_evolution <- function() {
  if (file.exists("resultados_optimizacion/solutions_history.csv")) {
    sol_data <- read.csv("resultados_optimizacion/solutions_history.csv")

    # Transformar datos para gráfico de parámetros
    param_data <- sol_data %>%
      select(ciclo, WM_factor, WF_factor, tiempo_inicio, duracion_M, delay_F, duracion_F) %>%
      pivot_longer(cols = -ciclo, names_to = "parametro", values_to = "valor")

    p <- ggplot(param_data, aes(x = as.factor(ciclo), y = valor, fill = parametro)) +
      geom_boxplot(alpha = 0.7) +
      labs(title = "Evolución de los Parámetros por Ciclo",
           x = "Ciclo", y = "Valor del Parámetro") +
      theme_minimal() +
      facet_wrap(~ parametro, scales = "free_y", ncol = 2) +
      theme(axis.text.x = element_text(angle = 45, hjust = 1))

    ggsave("resultados_optimizacion/evolucion_parametros.png", p, width = 12, height = 8)
    print(p)
  }
}

# Función de monitor mejorada
measure_time <- function(object, ...) {
  elapsed_time <- round(Sys.time() - start_time, 2)
  cat("Iteración:", object@iter,
      "| Tiempo:", elapsed_time, "segundos",
      "| Evaluaciones:", object@iter * object@popSize, "\n")
}

# ============================================================================
# EJECUCIÓN PRINCIPAL
# ============================================================================

start_time <- Sys.time()
current_generations <- 0
current_result <- NULL

cat("=== INICIANDO OPTIMIZACIÓN INCREMENTAL ===\n")
cat("Configuración: Población =", popsize, "| Incremento =", increment, "generaciones\n\n")

for (cycle in 1:(total_generations/increment)) {
  cat("\n", strrep("=", 50), "\n")
  cat("CICLO", cycle, "\n")
  cat("Generaciones:", current_generations, "->", current_generations + increment, "\n")
  cat(strrep("=", 50), "\n\n")

  cycle_start <- Sys.time()

  if (is.null(current_result)) {
    # Primera ejecución
    current_result <- rmoo::rmoo(
      type = "real-valued",
      algorithm = "NSGA-II",
      fitness = fitness_function,
      lower = c(0.0001, 0.0001, 1, 14, 7, 14),
      upper = c(2, 2, 300, 65, 67, 65),
      popSize = popsize,
      maxiter = increment,
      nObj = 2,
      pcrossover = 0.9,
      pmutation = 0.5,
      parallel = TRUE,
      monitor = measure_time
    )
  } else {
    # Continuar desde resultado anterior
    current_result <- rmoo::rmoo(
      type = "real-valued",
      algorithm = "NSGA-II",
      fitness = fitness_function,
      lower = c(0.0001, 0.0001, 1, 14, 7, 14),
      upper = c(2, 2, 300, 65, 67, 65),
      popSize = popsize,
      maxiter = increment,
      nObj = 2,
      pcrossover = 0.9,
      pmutation = 0.5,
      monitor = measure_time,
      parallel = TRUE,
      suggestions = current_result@population
    )
  }

  current_generations <- current_generations + increment
  cycle_time <- round(Sys.time() - cycle_start, 2)

  # Guardar resultados del ciclo
  cycle_data <- save_cycle_results(current_result, cycle, current_generations)
  results_history[[cycle]] <- current_result
  time_history[[cycle]] <- cycle_time

  # Resumen del ciclo
  cat("\n--- RESUMEN CICLO", cycle, "---\n")
  cat("Tiempo del ciclo:", cycle_time, "segundos\n")
  cat("Mejor Objetivo 1:", round(min(current_result@fitness[,1]), 4), "\n")
  cat("Mejor Objetivo 2:", round(min(current_result@fitness[,2]), 4), "\n")
  cat("Soluciones no dominadas:", nrow(current_result@fitness), "\n")

  # Generar gráficos actualizados
  cat("Generando gráficos...\n")
  plot_convergence()
  plot_pareto_fronts()
  plot_parameters_evolution()

  # Preguntar si continuar
  if (current_generations < total_generations) {
    cat("\n¿Continuar con las próximas", increment, "generaciones? (s/n): ")
    response <- readline()
    if (tolower(response) != "s") {
      cat("Optimización detenida por el usuario\n")
      break
    }
  }
}


In [ ]:
# ============================================================================
#GRÁFICOS PARA OPTIMIZACIÓN FRENTE DE PARETO Y PARÁMETROS
# ============================================================================



# Leer los datos
convergence_data <- read.csv("resultados_optimizacion/convergence_history.csv")
solutions_data <- read.csv("resultados_optimizacion/solutions_history.csv")


# 5A. Violin Plot de parámetros de liberación
param_data <- solutions_data %>%
  select(ciclo, WM_factor, WF_factor, tiempo_inicio, duracion_M, delay_F, duracion_F) %>%
  pivot_longer(cols = -ciclo, names_to = "parametro", values_to = "valor")

p5a <- ggplot(param_data, aes(x = as.factor(ciclo), y = valor, fill = parametro)) +
  geom_violin(alpha = 0.7, scale = "width", trim = TRUE) +
  geom_boxplot(width = 0.1, fill = "white", alpha = 0.7, outlier.shape = NA) +
  labs(
       x = "Ciclo",
       y = "Valor del Parámetro") +
  facet_wrap(~ parametro, scales = "free_y", ncol = 3) +
  scale_fill_viridis_d() +
  theme_minimal() +
  expand_limits(y = 0)+
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

print(p5a)

# Instalar el paquete si no lo tienes
if (!require(ggridges)) {
  install.packages("ggridges")
  library(ggridges)
}

# Versión minimalista con solo las curvas
p_minimal_density <- ggplot(param_data, aes(x = valor, color = parametro, fill = parametro)) +
  geom_density(alpha = 0.3, size = 1) +
  facet_wrap(~ parametro, ncol = 2, scales = "free") +
  labs(
    x = "Valor del Parámetro",
    y = "Densidad"
  ) +
  scale_color_viridis_d() +
  scale_fill_viridis_d() +
  theme_minimal() +
  theme(
    legend.position = "none",
    strip.text = element_text(face = "bold", size = 10),
    panel.grid.minor = element_blank()
  )

print(p_minimal_density)


# 5D. Violin Plot con puntos de datos
p5d <- ggplot(param_data, aes(x = as.factor(ciclo), y = valor, fill = parametro)) +
  geom_violin(alpha = 0.6, scale = "width", trim = TRUE) +
  geom_jitter(width = 0.2, alpha = 0.3, size = 0.8, color = "black") +
  stat_summary(fun = median, geom = "point", shape = 18, size = 2, color = "red") +
  labs(

       x = "Ciclo",
       y = "Valor del Parámetro") +
  facet_wrap(~ parametro, scales = "free_y", ncol = 3) +
  scale_fill_viridis_d() +
  theme_minimal() +
  expand_limits(y = 0)+
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

print(p5d)



p3b <- ggplot(solutions_data, aes(x = obj1, y = obj2)) +
  geom_point(alpha = 0.7, size = 1.5, color = "steelblue") +
  facet_wrap(~ ciclo, ncol = 4) +
  labs(title = "Frentes de Pareto Separados por Ciclo",
       x = "Función Objetivo 1",
       y = "Función Objetivo 2") +
  theme_minimal() +
  theme(strip.background = element_rect(fill = "lightgray"))
print(p3b)




p3f <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(aes(size = poblacion_WF), alpha = 0.7) +
  scale_size_continuous(name = "Población WF", range = c(1, 6)) +
  labs(
    x = "Machos infectados (WM)",
    y = "Hembras infectadas (WF)",
    color = "Ciclo") +
  scale_color_viridis_d() +
  theme_minimal() +

  scale_x_continuous(breaks = seq(0, 300000, by = 10000)) +  # Marcas cada 10000
  scale_y_continuous(breaks = seq(0, 300000, by = 10000))

print(p3f)



p3e <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(alpha = 0.8, size = 3) +
  labs(title = "Ciclo: {frame_time}",
       x = "Machos Infectados (WM)",
       y = "Hembras Infeactads (WF)",
       color = "Ciclo") +
  scale_color_viridis_d() +
  expand_limits(x = 0, y = 0) +
  scale_x_continuous(breaks = seq(0, 70000, by = 10000),
                     limits = c(0, 70000),
                     labels = function(x) ifelse(x %% 20000 == 0, x, "")) +  # Solo muestra cada 20000
  scale_y_continuous(breaks = seq(0, 70000, by = 10000),
                     limits = c(0, 70000),
                     labels = function(x) ifelse(x %% 20000 == 0, x, "")) +
  transition_time(ciclo) +
  ease_aes('linear')

# Crear y guardar animación
animacion <- animate(p3e,
                     nframes = length(unique(solutions_data$ciclo)) * 10,
                     fps = 20,
                     width = 800,
                     height = 600,
                     renderer = gifski_renderer())

anim_save("resultados_optimizacion/03e_pareto_animacion.gif", animacion)


p3f <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(aes(size = poblacion_WF), alpha = 0.7) +
  scale_size_continuous(name = "Población WF", range = c(1, 6)) +
  labs(
    x = "Machos infectados (WM)",
    y = "Hembras infectadas (WF)",
    color = "Ciclo") +
  scale_color_viridis_d() +
  theme_minimal() +
  coord_cartesian(xlim = c(0, max(solutions_data$obj1, na.rm = TRUE) * 1.1),
                  ylim = c(0, max(solutions_data$obj2, na.rm = TRUE) * 1.1)) +
  scale_x_continuous(breaks = seq(0, max(solutions_data$obj1, na.rm = TRUE), by = 10000)) +
  scale_y_continuous(breaks = seq(0, max(solutions_data$obj2, na.rm = TRUE), by = 10000))

print(p3f)


p3f <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(aes(size = poblacion_WF), alpha = 0.7) +
  scale_size_continuous(name = "Población WF", range = c(1, 6)) +
  labs(
    x = "Machos infectados (WM)",
    y = "Hembras infectadas (WF)",
    color = "Ciclo") +
  scale_color_viridis_d() +
  theme_minimal()

print(p3f)


p3 <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(alpha = 0.7, size = 2) +
  labs(title = "Frentes de Pareto por Ciclo",
       subtitle = "Evolución de las soluciones no dominadas",
       x = "Función Objetivo 1",
       y = "Función Objetivo 2",
       color = "Ciclo") +
  scale_color_viridis_d() +
  theme_minimal() +
  expand_limits(x = 0, y = 0) +
  scale_x_continuous(limits = c(0, 80000)) +
  scale_y_continuous(limits = c(0, 80000))

print(p3)

p3 <- ggplot(solutions_data, aes(x = obj1, y = obj2, color = as.factor(ciclo))) +
  geom_point(alpha = 0.8, size = 4) +  # Aumenté el tamaño a 4
  labs(
       x = "Función Objetivo 1",
       y = "Función Objetivo 2",
       color = "Ciclo") +
  scale_color_viridis_d() +
  theme_minimal() +
  expand_limits(x = 0, y = 0) +
  scale_x_continuous(limits = c(0, 80000), breaks = seq(0, 80000, by = 20000)) +
  scale_y_continuous(limits = c(0, 80000), breaks = seq(0, 80000, by = 20000)) +
  theme(legend.position = "right",
        plot.title = element_text(size = 14, face = "bold"),
        plot.subtitle = element_text(size = 12))

print(p3)



